This notebook contains the Bias correction (BC) process.

The user can choose either to correct precipitation (with the method BcsdPrecipitation(return_anoms=False), either temperature (with the method BcsdTemperature(return_anoms=False)).

The set of data is corrected following the following process: 

1) observation data and historic modeled data, both for the same period of time, are used to fit the model. Both set of data are at the emplacement of the closest NOAA meteorological station to the project
2) the data at the emplacement of the project is corrected by the fitted model. The data to correct is not at the same emplacement as the data use to fit the model. But, as explain previously, the data use to fit the model are at the emplecement of the closest NOAA station to the emplecement of the project

This notebook correct the data in the past, in order to compare historique bias corrected data with the historic observation data.

# User input

In [1]:
climate_var = 'tasmin' 

name_col_NOAA = 'TMIN'
name_col_mod_hist = 'Daily Minimum Near-Surface Air Temperature °C'
name_col_mod_proj = 'Daily Minimum Near-Surface Air Temperature °C'

unit='Celsius'
resolution = 'day'

start_y = 2030 #1970 # 2030
stop_y = 2065#2014 # 2065
# precipitation : 'pr',name_col_NOAA = 'PRCP', name_col_mod_hist='Mean of the daily precipitation rate mm/day', name_col_mod_proj = 'Mean of the daily precipitation rate mm_per_day', unit = 'mm_per_day', resolution = 'day'
# temperature : 'tas', name_col_NOAA = 'TAVG', name_col_mod_proj = 'Daily Near-Surface Air Temperature °C', unit = 'Celsius', resolution = 'day'
# maximum temperature : 'tasmax', name_col_NOAA = 'TMAX', name_col_mod_proj = 'Daily Maximum Near-Surface Air Temperature °C', unit = 'Celsius', resolution = 'day'
# minimum temperature : 'tasmin', name_col_NOAA = 'TMIN', name_col_mod_proj = 'Daily Minimum Near-Surface Air Temperature °C', unit = 'Celsius', resolution = 'day'


# Import packages and functions

In [2]:
# import modules and functions

import pandas as pd
import numpy as np
import os
import os.path

import matplotlib.pyplot as plt

# import data
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6
from Functions_ImportData import import_filtered_NOAA_obs
from Functions_ImportData import import_treat_obs_NOAA
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA
from Bias_correction_function import BC
from Bias_correction_function import treat_data_for_test
from Bias_correction_function import BCSD_Precipitation_return_anoms_to_apply
from Bias_correction_function import BCSD_Temperature_return_anoms_to_apply
from Functions_Indicators import add_year_month_season

# Import data

In [3]:
# import observation data
data_obs_NOAA_filtered=import_filtered_NOAA_obs()

In [4]:
# import historic modeled data
climate_var_NEX_GDDP_CMIP6_EmplacementStation=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA(climate_var, unit)

In [5]:
climate_var_NEX_GDDP_CMIP6_EmplacementStation

,Name station,Experiment,Model,Latitude,Longitude,Date,Daily Minimum Near-Surface Air Temperature °C,Year,Month,Season
0,"PEMBA, MZ",historical,ACCESS-CM2,-12.875,40.625,01-01-1970,23.652435,1970,Jan,Humid
1,"PEMBA, MZ",historical,ACCESS-CM2,-12.875,40.625,02-01-1970,23.662689,1970,Jan,Humid
2,"PEMBA, MZ",historical,ACCESS-CM2,-12.875,40.625,03-01-1970,23.334808,1970,Jan,Humid
3,"PEMBA, MZ",historical,ACCESS-CM2,-12.875,40.625,04-01-1970,23.928467,1970,Jan,Humid
4,"PEMBA, MZ",historical,ACCESS-CM2,-12.875,40.625,05-01-1970,23.834381,1970,Jan,Humid
...,...,...,...,...,...,...,...,...,...,...
690307,"BEIRA, MZ",historical,TaiESM1,-19.875,34.875,27-12-2014,32.294769,2014,Dec,Humid
690308,"BEIRA, MZ",historical,TaiESM1,-19.875,34.875,28-12-2014,31.915863,2014,Dec,Humid
690309,"BEIRA, MZ",historical,TaiESM1,-19.875,34.875,29-12-2014,33.483002,2014,Dec,Humid
690310,"BEIRA, MZ",historical,TaiESM1,-19.875,34.875,30-12-2014,34.706207,2014,Dec,Humid


In [6]:
# import data to correct
climate_var_NEX_GDDP_CMIP6_Project=import_treat_modeled_NEX_GDDP_CMIP6(climate_var,unit,resolution,1950,2100)

In [7]:
climate_var_NEX_GDDP_CMIP6_Project

,Name project,Experiment,Model,Latitude,Longitude,Date,Daily Minimum Near-Surface Air Temperature °C,Year,Month,Season
0,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,01-01-1950,24.757233,1950,Jan,Humid
1,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,02-01-1950,24.676422,1950,Jan,Humid
2,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,03-01-1950,24.394623,1950,Jan,Humid
3,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,04-01-1950,24.546448,1950,Jan,Humid
4,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,05-01-1950,20.226654,1950,Jan,Humid
...,...,...,...,...,...,...,...,...,...,...
8365555,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,27-12-2100,29.714172,2100,Dec,Humid
8365556,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,28-12-2100,29.372620,2100,Dec,Humid
8365557,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,29-12-2100,28.694733,2100,Dec,Humid
8365558,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,30-12-2100,27.819519,2100,Dec,Humid


In [8]:
# select years between which the data should be corrected. Usually, the data are not corrected for the same number of years over which the model was fitted
climate_var_NEX_GDDP_CMIP6_Project_to_correct=climate_var_NEX_GDDP_CMIP6_Project[climate_var_NEX_GDDP_CMIP6_Project['Year'].between(start_y,stop_y)]

# Apply BC

In [9]:
# preprare list of model to BC
list_model=list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Model']))
list_model.remove('NESM3') # this model has too much Nan to be interesting
list_model.remove('CMCC-CM2-SR5')
list_model.remove('TaiESM1')

In [10]:
df_bc_corrected=pd.DataFrame()
for name_project in list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Name project'])):
    # select data for one project
    climate_var_NEX_GDDP_CMIP6_Project_temp=climate_var_NEX_GDDP_CMIP6_Project_to_correct[climate_var_NEX_GDDP_CMIP6_Project_to_correct['Name project']==name_project]
    # depending on project, don t fit the model with the same station data
    if name_project == 'WTP_Mutua_EIB':
        name_station = 'BEIRA, MZ'
    if name_project =='Chimoio_WTP_EIB' or name_project =='Gorongosa_EIB':
        name_station = 'CHIMOIO, MZ'
    if name_project == 'Pemba_EIB':
        name_station = 'PEMBA, MZ'
    for scenario in list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Experiment'])):
        # select data for one scenario
        climate_var_NEX_GDDP_CMIP6_Project_temp_2=climate_var_NEX_GDDP_CMIP6_Project_temp[climate_var_NEX_GDDP_CMIP6_Project_temp['Experiment']==scenario]
        for model in list_model:
            print('For '+name_project+', '+scenario+' and '+model)
            # select data for one model
            climate_var_NEX_GDDP_CMIP6_Project_temp_3=climate_var_NEX_GDDP_CMIP6_Project_temp_2[climate_var_NEX_GDDP_CMIP6_Project_temp_2['Model']==model]
            # concatenate observed and historic modeled data in one dataframe df
            df = treat_data_for_test(data_obs_NOAA_filtered,name_col_NOAA,climate_var_NEX_GDDP_CMIP6_EmplacementStation,name_col_mod_hist,name_station,model)
            # apply method; choose method based on if climate_var is precipitation or temperature
            # X is the modeled historic set of data used to fit the model
            # y is the observed set of data used to fit the model
            # out is the product of the bias correction
            if climate_var == 'pr':
                (X_pcp,y_pcp,out)=BCSD_Precipitation_return_anoms_to_apply(df,climate_var_NEX_GDDP_CMIP6_Project_temp_3[['Date',name_col_mod_proj]].dropna().reset_index(drop=True))
            else:
                (X_temp,y_temp,out)=BCSD_Temperature_return_anoms_to_apply(df,climate_var_NEX_GDDP_CMIP6_Project_temp_3[['Date',name_col_mod_proj]].dropna().reset_index(drop=True))
            out = out.reset_index() # date is no longer the index
            out['Name project']=name_project # impose the name of the project in the dataframe
            out['Experiment']=scenario # impose the name of the scenario in the dataframe
            out['Model']=model # impose the name of the model in the dataframe
            df_bc_corrected=pd.concat([df_bc_corrected,out]) # concat the result to the other results

For Chimoio_WTP_EIB, ssp126 and CMCC-ESM2
For Chimoio_WTP_EIB, ssp126 and NorESM2-LM
For Chimoio_WTP_EIB, ssp126 and ACCESS-CM2
For Chimoio_WTP_EIB, ssp126 and CanESM5
For Chimoio_WTP_EIB, ssp126 and MPI-ESM1-2-LR
For Chimoio_WTP_EIB, ssp126 and MRI-ESM2-0
For Chimoio_WTP_EIB, ssp126 and MIROC6
For Chimoio_WTP_EIB, ssp126 and NorESM2-MM
For Chimoio_WTP_EIB, ssp126 and ACCESS-ESM1-5
For Chimoio_WTP_EIB, ssp126 and MPI-ESM1-2-HR
For Chimoio_WTP_EIB, ssp126 and BCC-CSM2-MR
For Chimoio_WTP_EIB, ssp370 and CMCC-ESM2
For Chimoio_WTP_EIB, ssp370 and NorESM2-LM
For Chimoio_WTP_EIB, ssp370 and ACCESS-CM2
For Chimoio_WTP_EIB, ssp370 and CanESM5
For Chimoio_WTP_EIB, ssp370 and MPI-ESM1-2-LR
For Chimoio_WTP_EIB, ssp370 and MRI-ESM2-0
For Chimoio_WTP_EIB, ssp370 and MIROC6
For Chimoio_WTP_EIB, ssp370 and NorESM2-MM
For Chimoio_WTP_EIB, ssp370 and ACCESS-ESM1-5
For Chimoio_WTP_EIB, ssp370 and MPI-ESM1-2-HR
For Chimoio_WTP_EIB, ssp370 and BCC-CSM2-MR
For Chimoio_WTP_EIB, ssp585 and CMCC-ESM2
For Chim

In [13]:
df_bc_corrected = df_bc_corrected.reset_index(drop=True) # reset index not to have several time the same index

In [14]:
df_bc_corrected['Date']=pd.to_datetime(df_bc_corrected[['Date']].Date,format='%Y-%d-%m')
df_bc_corrected['Date']=df_bc_corrected['Date'].astype(str)

In [15]:
df_bc_corrected

,Date,Daily Minimum Near-Surface Air Temperature °C,Name project,Experiment,Model
0,2030-01-01,22.029268,Chimoio_WTP_EIB,ssp126,CMCC-ESM2
1,2030-01-02,22.771426,Chimoio_WTP_EIB,ssp126,CMCC-ESM2
2,2030-01-03,21.293142,Chimoio_WTP_EIB,ssp126,CMCC-ESM2
3,2030-01-04,20.440858,Chimoio_WTP_EIB,ssp126,CMCC-ESM2
4,2030-01-05,20.455193,Chimoio_WTP_EIB,ssp126,CMCC-ESM2
...,...,...,...,...,...
2310579,2065-12-27,27.859478,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR
2310580,2065-12-28,29.396100,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR
2310581,2065-12-29,28.590368,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR
2310582,2065-12-30,27.320952,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR


In [16]:
# add month, year and season. Will be usefull to apply CRVA
df_bc_corrected = add_year_month_season(df_bc_corrected,'Date')

In [17]:
df_bc_corrected

,Date,Daily Minimum Near-Surface Air Temperature °C,Name project,Experiment,Model,Year,Month,Season
0,2030-01-01,22.029268,Chimoio_WTP_EIB,ssp126,CMCC-ESM2,2030,Jan,Humid
1,2030-01-02,22.771426,Chimoio_WTP_EIB,ssp126,CMCC-ESM2,2030,Jan,Humid
2,2030-01-03,21.293142,Chimoio_WTP_EIB,ssp126,CMCC-ESM2,2030,Jan,Humid
3,2030-01-04,20.440858,Chimoio_WTP_EIB,ssp126,CMCC-ESM2,2030,Jan,Humid
4,2030-01-05,20.455193,Chimoio_WTP_EIB,ssp126,CMCC-ESM2,2030,Jan,Humid
...,...,...,...,...,...,...,...,...
2310579,2065-12-27,27.859478,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR,2065,Dec,Humid
2310580,2065-12-28,29.396100,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR,2065,Dec,Humid
2310581,2065-12-29,28.590368,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR,2065,Dec,Humid
2310582,2065-12-30,27.320952,WTP_Mutua_EIB,ssp245,BCC-CSM2-MR,2065,Dec,Humid


# Export result as csv file

In [18]:
path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6-AllMoz\csv_file'

In [19]:
if not os.path.isdir(os.path.join(path,climate_var,climate_var+'_'+unit+'_'+resolution+'_'+str(start_y)+'-'+str(stop_y)+'_BiasCorrected')):
    os.makedirs(os.path.join(path,climate_var,climate_var+'_'+unit+'_'+resolution+'_'+str(start_y)+'-'+str(stop_y)+'_BiasCorrected'))

In [20]:
df_bc_corrected.to_csv(os.path.join(path,climate_var,climate_var+'_'+unit+'_'+resolution+'_'+str(start_y)+'-'+str(stop_y)+'_BiasCorrected',climate_var+'_'+unit+resolution+str(start_y)+'-'+str(stop_y)+'_BiasCorrected.csv'))